In [ ]:
#!del saved_models SVM_unigram*

In [ ]:
import glob
import numpy as np
import os
import pathlib
import pickle
import tensorflow as tf
from functools import partial
from pathlib import Path
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
import argparse

#my_parser = argparse.ArgumentParser()

#my_parser.add_argument('-start', help='starting time point', required=True)
#my_parser.add_argument('-window_size', help='size of window', required=True)
#my_parser.add_argument('-run_num', help='which number run this is', required=True)
#args = my_parser.parse_args()
#start = int(args.start)
#window_size = int(args.window_size)
#print(window_size)

#tf.enable_eager_execution()
print(tf.__version__)

BATCH_SIZE=512
n_electrodes = 64
n_timepoints = 176
window_shift = 4
window_size = 16
N_AVG = 1
category = "unigram_6class"
classes = [i for i in range(6)]
n_epoch = 3
model_type = "SVM"  
method = "tfr" # "numpy" # or "tfr"
run_num = 0
filename_stem = "unigram_6class"

print(f'n_avg = {N_AVG}, method = {method}')





# ------- TFRecord functionality START
def _float_feature_seq(eeg_channel):
    """ Convert sequence of EEG values to tf.train.FeatureList """
    feature_list = tf.train.FeatureList(feature=[
        tf.train.Feature(float_list=tf.train.FloatList(
            value=eeg_channel))])
    
    return feature_list

def _int_feature_scalar(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def encode_single_example(eeg_data: np.array, label: int):
    """ TFRecords: process single EEG trial as a SequenceExample"""
    #print(eeg_data.shape)
    eeg_data_flat = eeg_data.copy().reshape((-1, 1))
    
    fl_dict = {'eeg_data':_float_feature_seq(eeg_data_flat)}
    label_dict = {'label':  _int_feature_scalar(label)}
    
    label_features = tf.train.Features(feature=label_dict)
    
    #print(f'label_features : {label_features}')
    
    feature_lists = tf.train.FeatureLists(feature_list=fl_dict)
    protobuff = tf.train.SequenceExample(context=label_features,
                                         feature_lists=feature_lists)

    protobuff_serialised = protobuff.SerializeToString()
    
    return protobuff_serialised


def decode_single_example(serialised_example, start_window, end_window,
                           total_timepoints, n_electrodes):

    data_dim = total_timepoints * n_electrodes

    context_desc = {'label': tf.io.FixedLenFeature([], dtype=tf.int64)}
    feature_desc = {
        'eeg_data': tf.io.FixedLenSequenceFeature([data_dim], dtype=tf.float32)
    }

    context, data = tf.io.parse_single_sequence_example(
        serialized=serialised_example,
        context_features=context_desc,
        sequence_features=feature_desc,
        name='parsing_single_seq_example')

    data = data['eeg_data']
    data = tf.reshape(data, (n_electrodes, total_timepoints))
    data = tf.transpose(data, (1,0))
    data = tf.reshape(data, (total_timepoints, 64))
    
    print(f'data shape: {data.shape}')
    data = data[start_window:end_window, :] # Extract (potential sub-window)
    WINDOW_LENGTH = end_window - start_window
    data = tf.reshape(data, (1, WINDOW_LENGTH * n_electrodes))

    label = context['label']
    label = tf.cast(label, tf.int32)

    return data, label

def get_dataset(file_regex, folder, batch_size, repeat,
               start_window, end_window, total_timepoints,
               n_electrodes):
    
    decode_single_example_fn = partial(decode_single_example,
                                       start_window=start_window,
                                       end_window=end_window,
                                       total_timepoints=total_timepoints,
                                       n_electrodes=n_electrodes)
    
    files = list(pathlib.Path(folder).glob(file_regex))
    assert len(files) > 0, f"No files found for: {file_regex}"
    files = [str(x) for x in files]
    print(f'Found files: {files}')
    
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads=1)
    dataset = dataset.map(decode_single_example_fn, num_parallel_calls=1)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.shuffle(BATCH_SIZE)
    dataset = dataset.repeat(repeat)

    return dataset
# ------- TFRecord functionality END




# Function to run model
def run_model_tfr(train_ds, dev_ds, test_ds):
    
    all_dev_scores = [] # to store all window-level dev scores for this  run
    
    for start_loc in range(0, n_timepoints, window_shift):
        
        end_loc = start_loc + window_size
        window_begin = start_loc * n_electrodes
        window_end = end_loc * n_electrodes
        print(f'Running window: {start_loc} to {end_loc}')
        
        score = 0
        best_score = 0
        best_global_dev = -1
        best_test = 0
        SGD_clf = SGDClassifier(loss='hinge', tol=1e-3, alpha=0.75)

        for i, (X, y) in enumerate(train_ds):

            #print('Printing X shape and y shape [before]')
            #print(tf.shape(X))
            #print(tf.shape(y))
            #print(np.shape(X))
            #print(np.shape(y))
            
            #print('X shape before selecting window', X.shape)
            # (512, 1, 1024)

            #X = tf.squeeze(X) # remove superfluous dimension
            X = X[:, 0, window_begin:window_end] # get right window
            
            #print('X shape after selecting window', X.shape)
            # (512, 1024)

            #print('Printing X shape and y shape [after squeeze before slice]')
            #print(tf.shape(X))
            #print(tf.shape(y))
            #print(np.shape(X))
            #print(np.shape(y))

            #X = tf.slice(X, begin=[window_begin], size=[window_end-window_begin])

            #print('Printing X shape and y shape [after]')
            #print(tf.shape(X))
            #print(tf.shape(y))
            #print(np.shape(X))
            #print(np.shape(y))
            
            SGD_clf.partial_fit(X, y, classes=classes)

            dev_scores = []
            for i, (X_dev, y_dev) in enumerate(dev_ds):

                #X_dev = tf.squeeze(X_dev)
                #X_dev = X_dev[:, window_begin:window_end]
                
                
                X_dev = X_dev[:, 0, window_begin:window_end] # get right window
                

                dev_score = SGD_clf.score(X_dev, y_dev)
                dev_scores.append(dev_score)
              
            dev_score = np.mean(dev_scores)
            #print(f'{model_type} : epoch ({i}), dev score = {dev_score*100}%')

            if dev_score > best_score:
                best_score = score
                pickle.dump(SGD_clf,
                    open(os.path.join("saved_models",
                      "{}_avg{}_start{}_end{}_run{}.pkl".format(category, N_AVG,
                                                                start_loc, end_loc, run_num)), "wb"))

            if dev_score > best_global_dev:
                best_global_dev = score
                #best_test = SGD_clf.score(test_data, test_labels)

        model = pickle.load(
            open(os.path.join("saved_models",
                              "{}_avg{}_start{}_end{}_run{}.pkl".format(category, N_AVG,
                                                                        start_loc, end_loc, run_num)), "rb"))
        
        
        dev_scores = []
        for i, (X_dev, y_dev) in enumerate(dev_ds):
            
            #print('X_dev shape before selecting window', X_dev.shape)              
            
            #X_dev = tf.squeeze(X_dev)
            #X_dev = X_dev[:, window_begin:window_end]
            X_dev = X_dev[:, 0, window_begin:window_end] # get right window
            
            #print('X_dev shape after selecting window', X_dev.shape)
              
            dev_score = model.score(X_dev, y_dev)
            dev_scores.append(dev_score)
              
        dev_score = np.mean(dev_scores)
        print(f'({start_loc}-{end_loc}): score: {dev_score*100}%')
        all_dev_scores.append(dev_score)


    print(all_dev_scores)
    
    return best_global_dev, best_test, score, model, all_dev_scores

if method == "tfr":
    print("Method = ", method)
    get_dataset_fn = partial(get_dataset,
                            folder=".",
                            batch_size=BATCH_SIZE,
                            start_window=0, 
                            end_window=176,
                            total_timepoints=176,
                            repeat=4,
                            n_electrodes=64)

    folder = "./tfrecords_numpy_matching/"
    train_ds = get_dataset_fn(folder=folder,
                            file_regex=f"unigram_6class_avg{N_AVG}_train*.tfrecords")

    dev_ds = get_dataset_fn(folder=folder,
                            file_regex=f"unigram_6class_avg{N_AVG}_dev*.tfrecords")

    test_ds = get_dataset_fn(folder=folder,
                            file_regex=f"unigram_6class_avg{N_AVG}_test*.tfrecords")

    best_global_dev, best_test, score, SGD_clf, all_dev_scores = run_model_tfr(train_ds, dev_ds, test_ds)

elif method == "numpy":
    print("Method = ", method)
    run_model_numpy(train_data, train_labels,dev_data, dev_labels,test_data, test_labels)
else:
    raise Exception("Method not understood")

print("Final Scores")
print(all_dev_scores)

2.6.0
n_avg = 1, method = tfr
Method =  tfr
Found files: ['tfrecords_numpy_matching\\unigram_6class_avg1_train_file0.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file1.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file10.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file11.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file12.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file13.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file14.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file15.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file16.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file17.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file18.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file19.tfrecords', 'tfrecords_numpy_matching\\unigram_6class_avg1_train_file2.tfrecords', 'tfrecord